In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml.feature import NGram, Tokenizer

In [2]:
spark = (
    SparkSession.builder
                .appName("Stack Overflow Data Wrangling")
                .config("spark.jars", "/home/adutwumwaah/Desktop/Blossom/jars/postgresql-42.2.8.jar") 
                .getOrCreate()
)



Loading Dataset

In [3]:
questions = spark.read.csv("questions.csv", header = True, inferSchema = True,escape='"', multiLine=True)
answers = spark.read.csv("answers.csv", header = True, inferSchema = True,escape='"', multiLine=True)
users = spark.read.csv("users.csv", header = True, inferSchema = True,escape='"', multiLine=True)

Renaming columns

In [4]:
questions = questions.withColumnRenamed('id','qid')
questions = questions.withColumnRenamed('body', 'qbody')
questions = questions.withColumnRenamed('score', 'qscore')
questions = questions.withColumnRenamed('comment_count', 'qcomment_count')
questions = questions.withColumnRenamed('created_at', 'qcreated_at')
answers = answers.withColumnRenamed('created_at', 'acreated_at')
answers = answers.withColumnRenamed('id', 'aid')
answers = answers.withColumnRenamed('user_id', 'auser_aid')

In [5]:
questions.columns

['qid',
 'user_id',
 'title',
 'qbody',
 'accepted_answer_id',
 'qscore',
 'view_count',
 'qcomment_count',
 'qcreated_at']

In [6]:
answers.columns

['aid',
 'auser_aid',
 'question_id',
 'body',
 'score',
 'comment_count',
 'acreated_at']

In [7]:
users.columns

['id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'created_at',
 'updated_at']

#Extract the country and city into new columns

In [8]:
users = users\
                .withColumn('city', F.split(users['location'], ',')[0])\
                .withColumn('country', F.split(users['location'],',')[2])


Selecting users from only one country(India)

In [9]:
users.registerTempTable('users')
users = spark.sql("select * from users where country like '%India%' ")

In [10]:
users.count()

14923

In [11]:
#joining the resulting users df with questions 

df = users.join(questions, users.id == questions.user_id)

In [12]:
df_1 = df.filter(df['view_count'] > 19)

In [13]:
#Joining answers dataset
df_2 = df_1.join(answers, df_1.id == answers.auser_aid)

In [14]:
df_2.columns

['id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'created_at',
 'updated_at',
 'city',
 'country',
 'qid',
 'user_id',
 'title',
 'qbody',
 'accepted_answer_id',
 'qscore',
 'view_count',
 'qcomment_count',
 'qcreated_at',
 'aid',
 'auser_aid',
 'question_id',
 'body',
 'score',
 'comment_count',
 'acreated_at']

In [15]:
df_2.select("updated_at").take(10)

[Row(updated_at=datetime.datetime(2019, 9, 28, 16, 55, 54)),
 Row(updated_at=datetime.datetime(2019, 9, 28, 16, 55, 54)),
 Row(updated_at=datetime.datetime(2019, 9, 28, 16, 55, 54)),
 Row(updated_at=datetime.datetime(2019, 9, 28, 16, 55, 54)),
 Row(updated_at=datetime.datetime(2019, 5, 15, 8, 0, 50)),
 Row(updated_at=datetime.datetime(2019, 10, 13, 6, 4, 6)),
 Row(updated_at=datetime.datetime(2019, 10, 13, 5, 15, 34)),
 Row(updated_at=datetime.datetime(2019, 10, 13, 5, 15, 34)),
 Row(updated_at=datetime.datetime(2019, 10, 13, 5, 15, 34)),
 Row(updated_at=datetime.datetime(2019, 10, 13, 5, 15, 34))]

In [37]:
df_2.write.format("jdbc").options(
    url='jdbc:postgresql://localhost/postgres',
    driver='org.postgresql.Driver',
    user='postgres',
    password='devinvogue',
    dbtable='stackoverflow_filtered.results').save(mode='append')


 Section 3
 8. Differences between View and Materialized View

1. View is a virtual table that is created for display purposes and it is not stored on the disk whilst a materialized view is a physical copy of the base table created and stored on a disk.

2. View is updated anytime the virtual table is viewd whilst the materilized view is updated manually.

3. View has slow processing whilst materialized view has fast processing.
